# Introducción

Supongamos que queremos calcular los gradientes de la función $f(x,y) = x^2y+y+2$ con respecto a los argumentos $x$ y $y$:

In [4]:
def f(x,y):
    return x*x*y + y + 2

Una opción es obtener las derivadas analíticamente:

$\frac{\partial f}{\partial x} = 2xy$

$\frac{\partial f}{\partial y} = x^2 + 1$

In [2]:
def df(x,y):
    return 2*x*y, x*x + 1

Por ejemplo, $\frac{\partial f}{\partial x}(3,4) = 24$ y $\frac{\partial f}{\partial y}(3,4) = 10$.

In [3]:
df(3, 4)

(24, 10)

También podemos obtener las ecuaciones para las segundas derivadas (*hessianas*):

$\dfrac{\partial^2 f}{\partial x \partial x} = \dfrac{\partial (2xy)}{\partial x} = 2y$

$\dfrac{\partial^2 f}{\partial x \partial y} = \dfrac{\partial (2xy)}{\partial y} = 2x$

$\dfrac{\partial^2 f}{\partial y \partial x} = \dfrac{\partial (x^2 + 1)}{\partial x} = 2x$

$\dfrac{\partial^2 f}{\partial y \partial y} = \dfrac{\partial (x^2 + 1)}{\partial y} = 0$

En $x=3$ y $y=4$ estas hessianas son respectivamente $8$, $6$, $6$, $0$.

In [4]:
def d2f(x, y):
    return [2*y, 2*x], [2*x, 0]

In [5]:
d2f(3, 4)

([8, 6], [6, 0])

Para una red neuronal profunda (con muchas capas) es imposible evaluar las derivadas en esta manera. Vamos a buscar como automatizar este proceso.

# Derivación numérica

Diferencias finítas: aproximamos los gradientes usando la definición de la derivada

$$\frac{\partial f}{\partial x} = \lim_{\epsilon \to 0} \frac{f(x+\epsilon,y) - f(x,y)}{\epsilon}$$ 

Hay una definición similar para $\partial f/\partial y$.

In [1]:
def gradientes(func, vars_lista, eps=0.0001):
    derivadas_parciales = []
    func_eval = func(*vars_lista)
    for idx in range(len(vars_lista)):
        vars_epsilon = vars_lista[:]
        vars_epsilon[idx] += eps
        func_eval_epsilon = func(*vars_epsilon)
        derivada = (func_eval_epsilon - func_eval) / eps
        derivadas_parciales.append(derivada)
    return derivadas_parciales

In [2]:
def df(x, y):
    return gradientes(f, [x, y])

In [5]:
df(3, 4)

[24.000400000048216, 10.000000000047748]

Ahora, calculamos los elementos del jacobiano:

In [6]:
def dfdx(x, y):
    return gradientes(f, [x,y])[0]

def dfdy(x, y):
    return gradientes(f, [x,y])[1]

In [7]:
dfdx(3., 4.), dfdy(3., 4.)

(24.000400000048216, 10.000000000047748)

In [8]:
def d2f(x, y):
    return [gradientes(dfdx, [x, y]), gradientes(dfdy, [x, y])]

In [9]:
d2f(3, 4)

[[7.999999951380232, 6.000099261882497],
 [6.000099261882497, -1.4210854715202004e-06]]

Funciona, pero con aproximaciones. Además, para calcular los gradientes de una función con respecto a $n$ variables requiere $n$ evaluaciones de dicha función. En redes neuronales profundas hay miles de parámetros, así que este método es demasiado lento.

# Implementación de un gráfo computacional (de juguete)

Ahora vamos a definir nuestro propio código para implementar el procedimiento de AutoDiff.

In [12]:
class Const(): # Suponiendo Python 3!!
    def __init__(self, valor):
        self.valor = valor
    def evaluar(self):
        return self.valor
    def __str__(self):
        return str(self.valor)
    
class Var():
    def __init__(self, nombre, ini_valor=0):
        self.valor = ini_valor
        self.nombre = nombre
    def evaluar(self):
        return self.valor
    def __str__(self):
        return self.nombre
    
class OperadorBinario():
    def __init__(self, a, b):
        self.a = a
        self.b = b
        
class Suma(OperadorBinario):
    def evaluar(self):
        return self.a.evaluar() + self.b.evaluar()
    def __str__(self):
        return "{} + {}".format(self.a, self.b)
    
class Prod(OperadorBinario):
    def evaluar(self):
        return self.a.evaluar() * self.b.evaluar()
    def __str__(self):
        return "({}) * ({})".format(self.a, self.b)

Ahora podemos contruir un gráfo computacional para representar la función $f$:

In [15]:
x = Var("x")
y = Var("y")
f = Suma(Prod(Prod(x, x), y), Suma(y, Const(2))) # f(x,y) = x^2 y + y + 2

Podemos "ejecutar" el gráfo para calcular $f$ en cualquier punto, por ejemplo $f(3,4)$

In [16]:
x.valor = 3
y.valor = 4
f.evaluar()

42

# Calculando gradientes

Los métodos de AutoDiff que veremos abajo están todos basados en la *regla de cadena*.

Supongamos que tenemos dos funciones $u$ y $v$, y las aplicamos secuencialmente a una entrada $x$, para obtener el resultado $z$. Entonces, $z = v(u(x))$. Podemos escribir esta composición de funciones así:

$$z = v(s) \quad \quad s = u(x)$$

Ahora, podemos aplicar la regla de cadena para obtener la derivada de $z$ con respecto a $x$:

$$\frac{\partial z}{\partial x} = \frac{\partial s}{\partial x} \cdot \frac{\partial z}{\partial s}$$

Se puede generalizar este resultado inmediatemente al caso de tener una secuencia de $n$ funciones: $s_1,s_2,\ldots,s_n$:

$$\frac{\partial z}{\partial x} = \frac{\partial s_1}{\partial x} \cdot \frac{\partial s_2}{\partial s_1} \cdot \frac{\partial s_3}{\partial s_2} \cdot \cdots \cdot \frac{\partial s_{n-1}}{\partial s_{n-2}} \cdot \frac{\partial s_n}{\partial s_{n-1}} \cdot \frac{\partial z}{\partial s_n}$$

En modo hacia adelante de AutoDiff (*forward mode*) el algoritmo calcula estos términos en el mismo orden en que están escritos arriba, de la izquierda hacia la derecha, comenzando con $\partial s_1/\partial x$.

En modo hacia atrás (*reverse mode*) el algoritmo va de la derecha hacia la izquierda. Es decir, comienza con $\partial z/\partial s_n$.

### Ejemplo (*forward mode*)

Queremos calcular la derivada de la función

$$z(x) = \sin(x^2)$$

en el punto $x = 3$ ocupando AutoDiff en modo hacia adelante. El algoritmo calculará primero la derivada

$$\frac{\partial s_1}{\partial x} = \frac{\partial x^2}{\partial x} = 2x = 6$$

Después calculará

$$\frac{\partial z}{\partial x} = \frac{\partial s_1}{\partial x} \cdot \frac{\partial z}{\partial s_1} = 6 \cdot \frac{\partial \sin(s_1)}{\partial s_1} = 6 \cdot \cos(s_1) = 6 \cdot \cos(3^2) \approx -5.46$$

Verificamos este resultado usando la función `gradientes()` que vimos antes:

In [17]:
from math import sin

def z(x):
    return sin(x**2)

In [18]:
gradientes(z, [3])

[-5.46761419430053]

### Ejemplo (*reverse mode*)

Ahora hacemos lo mismo con AutoDiff hacia atrás. El algoritmo calculará

$$\frac{\partial z}{\partial s_1} = \frac{\partial \sin(s_1)}{\partial s_1} = \cos(s_1) = \cos(3^2) \approx -0.91$$

Después calculará

$$\frac{\partial z}{\partial x} = \frac{\partial s_1}{\partial x} \cdot \frac{\partial z}{\partial s_1} \approx \frac{\partial s_1}{\partial x} \cdot -0.91 = \frac{\partial x^2}{\partial x} \cdot -0.91 = 2x \cdot -0.91 = 6 \cdot -0.91 = -5.46$$

Con una sola entrada y salida, ambos métodos requieren el mismo número de cálculaciones. Pero en el caso de tener varias entradas y salidas, los dos modos pueden deferir muchísimo en su rendimiento.

En el caso de tener muchas entradas, los términos hacia la derecha son requeridos para calcular las derivadas con respecto a cada entrada. Por lo tanto es mejor calcular esas derivadas primero (*reverse mode*). Así podemos calcular los términos a la derecha una sóla vez y usarlos para calcular todas las derivadas parciales.

En el caso de tener muchas salids, se puede calcular los términos a la izquierda primero, y usarlos para calcular las derivadas parciales con respecto a las salidas. Este corresponde a *forward mode*.

En *deep learning* hay miles de parámetros (las entradas del algoritmo) y pocas salidas. De hecho, típicamente hay una sola salida: el valor de la función de coste!

Por tal razón, el modo hacia atrás (*reverse mode*) es usado en TensorFlow.

#### Pase hacia adelante primero

Hay un detalle con respecto a AutoDiff con *reverse mode*. Para calcular $\partial s_{i+1}/\partial s_i$ necesitamos el valor de $s_i$, pero para calcular $s_i$ necesitamos $s_{i-1}$, que requiere $s_{i-1}$, etc.

Por lo tanto, necesitamos una primera pase por la red hacia adelante para calcular y guardar los valores de $s_1,s_2,s_3,\ldots,s_{n}$. A veces este es problemático para el RAM...

## AutoDiff modo hacia adelante (*forward mode*)

In [19]:
Const.gradiente = lambda self, var: Const(0)
Var.gradiente = lambda self, var: Const(1) if self is var else Const(0)
Suma.gradiente = lambda self, var: Suma(self.a.gradiente(var), self.b.gradiente(var))
Prod.gradiente = lambda self, var: Suma(Prod(self.a, self.b.gradiente(var)), Prod(self.a.gradiente(var), self.b))

In [20]:
x = Var(nombre="x", ini_valor=3.)
y = Var(nombre="y", ini_valor=4.)

In [21]:
f = Suma(Prod(Prod(x, x), y), Suma(y, Const(2))) # f(x,y) = x^2y + y + 2

In [22]:
dfdx = f.gradiente(x) # 2xy
dfdy = f.gradiente(y) #x^2 + 1

In [23]:
dfdx.evaluar(), dfdy.evaluar()

(24.0, 10.0)

Ya que la salida del método `gradiente()` es totalmente simbólica, podemos calcular derivadas de cualquier orden:

In [24]:
d2fdxdx = dfdx.gradiente(x) #2y
d2fdxdy = dfdx.gradiente(y) #2x
d2fdydx = dfdy.gradiente(x) #2x
d2fdydy = dfdy.gradiente(y) #0

In [26]:
[[d2fdxdx.evaluar(), d2fdxdy.evaluar()],
 [d2fdydx.evaluar(), d2fdydy.evaluar()]]

[[8.0, 6.0], [6.0, 0.0]]

El resultado es exacto.

## AutoDiff de *forward mode* con números duales

Otra forma de implementar el algoritmo es con números duales. Tiene la forma $z = a+b\epsilon$ donde $a$ y $b$ son números reales y $\epsilon$ es un número positivo infinitesimal, más pequeño que cualquier otro número, tal que $\epsilon^2 = 0$.

Por el teorema de Taylor, tenemos

$$f(x+\epsilon) = f(x) + \frac{df}{dx}\epsilon$$

Así que calculando $f(x+\epsilon)$ obtenemos $f(x)$ y su derivada.

Reglas aritméticas de números duales:

**Suma**

$$(a_1+b_1\epsilon) + (a_2+b_2\epsilon) = (a_1 + a_2) + (b_1+b_2)\epsilon$$

**Resta**

$$(a_1+b_1\epsilon) - (a_2+b_2\epsilon) = (a_1 - a_2) + (b_1-b_2)\epsilon$$

**Producto**

$$(a_1+b_1\epsilon) \times (a_2+b_2\epsilon) = (a_1a_2) + (a_1b_2+a_2b_1)\epsilon + b_1b_2\epsilon^2 = (a_1a_2) + (a_1b_2+a_2b_1)\epsilon$$

**División**

$$\frac{a_1+b_1\epsilon}{a_2+b_2\epsilon} = \frac{a_1+b_1\epsilon}{a_2+b_2\epsilon} \cdot \frac{a_2-b_2\epsilon}{a_2-b_2\epsilon} = \frac{a_1a_2 + (b_1a_2-a_1b_2)\epsilon - b_1b_2\epsilon^2}{a_2^2 + (a_2b_2-a_2b_2)\epsilon - b_2^2\epsilon} = \frac{a_1}{a_2} + \frac{a_1b_2-b_1a_2}{a_2^2}\epsilon$$

**Potencia**

$$(a+b\epsilon)^n = a^n + (na^{n-1}b)\epsilon$$

etc.

In [40]:
class NumeroDual(object):
    def __init__(self, valor=0.0, eps=0.0):
        self.valor = valor
        self.eps = eps
    def __add__(self, b):
        return NumeroDual(self.valor + self.to_dual(b).valor,
                          self.eps + self.to_dual(b).eps)
    def __radd__(self, a):
        return self.to_dual(a).__add__(self)
    def __mul__(self, b):
        return NumeroDual(self.valor * self.to_dual(b).valor,
                          self.eps * self.to_dual(b).valor + self.valor * self.to_dual(b).eps)
    def __rmul__(self, a):
        return self.to_dual(a).__mul__(self)
    def __str__(self):
        if self.eps:
            return "{:.1f} + {:.1f}ε".format(self.valor, self.eps)
        else:
            return "{:.1f}".format(self.valor)
    def __repr__(self):
        return str(self)
    @classmethod
    def to_dual(cls, n):
        if hasattr(n, "valor"):
            return n
        else:
            return cls(n)

$3 + (3+4\epsilon) = 6+4\epsilon$

In [41]:
3 + NumeroDual(3, 4)

6.0 + 4.0ε

$(3 + 4ε)\times(5 + 7ε)$ = $3 \times 5 + 3 \times 7ε + 4ε \times 5 + 4ε \times 7ε$ = $15 + 21ε + 20ε + 28ε^2$ = $15 + 41ε + 28 \times 0$ = $15 + 41ε$

In [29]:
NumeroDual(3, 4) * NumeroDual(5, 7)

15.0 + 41.0ε

In [30]:
x.valor = NumeroDual(3.0)
y.valor = NumeroDual(4.0)

In [31]:
f.evaluar()

42.0

La clase para número duales funciona sin problemas con la clase del gráfo computacional.

Ahora, calculamos las derivadas parciales de $f$ con respecto a $x$ y $y$ en $x=3$, $y=4$ usando los numeros duales.

In [32]:
x.valor = NumeroDual(3.0, 1.0) #3 + ε
y.valor = NumeroDual(4.0)

In [33]:
dfdx = f.evaluar().eps

In [34]:
dfdx

24.0

In [35]:
x.valor = NumeroDual(3.0) #3
y.valor = NumeroDual(4.0, 1.0) # 4 + ε

In [36]:
dfdy = f.evaluar().eps

In [37]:
dfdy

10.0

En nuestra implementación estamos limitados a derivadas de primer orden

## AutoDiff hacia atrás (*reverse mode*)

In [42]:
class Const():
    def __init__(self, valor):
        self.valor = valor
    def evaluar(self):
        return self.valor
    def backpropagate(self, gradiente):
        pass
    def __str__(self):
        return str(self.valor)
    
class Var():
    def __init__(self, nombre, ini_valor=0):
        self.valor = ini_valor
        self.nombre = nombre
        self.gradiente = 0
    def evaluar(self):
        return self.valor
    def backpropagate(self, gradiente):
        self.gradiente += gradiente
    def __str__(self):
        return self.nombre
    
class OperadorBinario():
    def __init__(self, a, b):
        self.a = a
        self.b = b
        
class Suma(OperadorBinario):
    def evaluar(self):
        self.valor = self.a.evaluar() + self.b.evaluar()
        return self.valor
    def backpropagate(self, gradiente):
        self.a.backpropagate(gradiente)
        self.b.backpropagate(gradiente)
    def __str__(self):
        return "{} + {}".format(self.a, self.b)
    
class Prod(OperadorBinario):
    def evaluar(self):
        self.valor = self.a.evaluar() * self.b.evaluar()
        return self.valor
    def backpropagate(self, gradiente):
        self.a.backpropagate(gradiente * self.b.valor)
        self.b.backpropagate(gradiente * self.a.valor)
    def __str__(self):
        return "({}) * ({})".format(self.a, self.b)

In [43]:
x = Var("x", ini_valor=3)
y = Var("y", ini_valor=4)

In [44]:
f = Suma(Prod(Prod(x, x), y), Suma(y, Const(2))) #f(x,y) = x^2y + y + 2

In [45]:
resultado = f.evaluar()

In [46]:
f.backpropagate(1.0)

In [47]:
print(f)

((x) * (x)) * (y) + y + 2


In [48]:
resultado

42

In [49]:
x.gradiente

24.0

In [50]:
y.gradiente

10.0

En este implementación las salidas son números, no expresiones simbólicas, así que no podemos obtener derivadas de mayor orden.

Podríamos modificar los métodos de `backpropagate()` para devolver expresiones simbólicas en vez de valores (por ejemplo `return Suma(2,3)` en vez de `5`). Con eso sería posible calcular derivadas de cualquier orden.

TensorFlow hace exactamente eso.

## AutoDiff de *reverse mode* con TensorFlow

In [51]:
import tensorflow as tf

2023-09-05 15:37:04.309064: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-05 15:37:12.646669: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [52]:
x = tf.Variable(3.)
y = tf.Variable(4.)

2023-09-05 15:37:20.516286: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-05 15:37:20.832173: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [53]:
with tf.GradientTape() as tape:
    f = x*x*y + y + 2

In [54]:
jacobianos = tape.gradient(f, [x, y])

In [55]:
jacobianos

[<tf.Tensor: shape=(), dtype=float32, numpy=24.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]

Podemos calcular derivadas de cualquier orden, porque todo es simbólico.

In [60]:
x = tf.Variable(3.)
y = tf.Variable(4.)

In [61]:
with tf.GradientTape(persistent=True) as tape:
    f = x*x*y + y + 2
    df_dx, df_dy = tape.gradient(f, [x, y])

In [62]:
d2f_d2x, d2f_dydx = tape.gradient(df_dx, [x, y])
d2f_dxdy, d2f_d2y = tape.gradient(df_dy, [x, y])
del tape

In [63]:
hessianos = [[d2f_d2x, d2f_dydx], [d2f_dxdy, d2f_d2y]]

In [64]:
hessianos

[[<tf.Tensor: shape=(), dtype=float32, numpy=8.0>,
  <tf.Tensor: shape=(), dtype=float32, numpy=6.0>],
 [<tf.Tensor: shape=(), dtype=float32, numpy=6.0>, None]]

Notar aquí que la última derivada $\partial^2 f/\partial y^2 = 0$. En TensorFlow esta valor está representado por `None`.